In [2]:
import os
from credentials import Credentials
creds = Credentials()
os.environ["OPENAI_API_KEY"] = creds.openai_api_key

In [8]:
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.memory import ConversationBufferMemory
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_mcp_adapters.client import MultiServerMCPClient

In [9]:
import warnings
warnings.filterwarnings("ignore")

In [1]:
from langchain.tools import tool
import requests

@tool
def save_conversation_tool(conversation: str) -> str:
    """Сохраняет переданный текст с помощью MCP."""
    try:
        response = requests.post(
            "http://localhost:8000/tools/save_conversation",
            json={"conversation": conversation}
        )
        return response.json().get("result", "Не удалось сохранить разговор.")
    except Exception as e:
        return f"Ошибка при обращении к MCP: {str(e)}"

In [10]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

# Инициализация модели
llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)

# Определение пользовательского инструмента
tools = [save_conversation_tool]

# Создание агента с использованием ReAct
memory = ConversationBufferMemory(memory_key="chat_history")
agent = initialize_agent(
    tools=tools,
    llm=llm,
    memory = memory,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

In [13]:
# Пример использования
agent.invoke({"input": "Пожалуйста, сохрани следующий текст: 'Текст, который нужно сохранить'"})



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: save_conversation_tool
Action Input: Текст, который нужно сохранить
```
Observation: Ошибка при обращении к MCP: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /tools/save_conversation (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x708c8c5ee0b0>: Failed to establish a new connection: [Errno 111] Connection refused'))
Thought:Thought: Do I need to use a tool? No
AI: К сожалению, сейчас не удаётся сохранить текст из-за технической ошибки при подключении к сервису. Могу ли я помочь вам с чем-то ещё?

> Finished chain.


{'input': "Пожалуйста, сохрани следующий текст: 'Текст, который нужно сохранить'",
 'chat_history': '',
 'output': 'К сожалению, сейчас не удаётся сохранить текст из-за технической ошибки при подключении к сервису. Могу ли я помочь вам с чем-то ещё?'}